In [2]:
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
import glob
import pandas as pd
import numpy as np
from scipy import interpolate
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from google.colab import drive
import pickle
from scipy.io import wavfile
from scipy.signal import decimate
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

ModuleNotFoundError: No module named 'tensorflow'

In [1]:
# Network Function
def Multi_Modal():

  # Define input layers for each head
  input_w = Input(shape = (25,24,1))
  input_p = Input(shape = (100,21,1))
  input_s = Input(shape = (25,32,1))

  # Encoders
  conv1_w = Conv2D(64, (3, 3), activation='relu')(input_w)
  pool1_w = MaxPooling2D((2, 2))(conv1_w)
  conv2_w = Conv2D(32, (3, 3), activation='relu')(pool1_w)
  pool2_w = MaxPooling2D((2, 2))(conv2_w)
  flatten_w = Flatten()(pool2_w)

  conv1_p = Conv2D(64, (3, 3), activation='relu')(input_p)
  pool1_p = MaxPooling2D((2, 2))(conv1_p)
  conv2_p = Conv2D(32, (3, 3), activation='relu')(pool1_p)
  pool2_p = MaxPooling2D((2, 2))(conv2_p)
  flatten_p = Flatten()(pool2_p)

  conv1_s = Conv2D(64, (3, 3), activation='relu')(input_s)
  pool1_s = MaxPooling2D((2, 2))(conv1_s)
  conv2_s = Conv2D(32, (3, 3), activation='relu')(pool1_s)
  pool2_s = MaxPooling2D((2, 2))(conv2_s)
  flatten_s = Flatten()(pool2_s)


  # Combine the outputs of hidden layers
  combined = concatenate([flatten_w, flatten_p, flatten_s])

  # Classifier
  # 64-32
  hidden1 = Dense(64, activation='relu')(combined)
  hidden2 = Dense(32, activation='relu')(hidden1)

  # Add more layers for further processing
  output = Dense(9, activation='softmax')(hidden2)

  model = Model(inputs=[input_w, input_p,input_s], outputs=output)
  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
# Network Function
def Uni_Modal(modality):

  if modality == 0:
    input = Input(shape = (100,21,1))
  elif modality == 1:
    input = Input(shape = (25,24,1))
  elif modality == 2:
    input = Input(shape = (25,32,1))

  # Encoders
  conv1 = Conv2D(64, (3, 3), activation='relu')(input)
  pool1 = MaxPooling2D((2, 2))(conv1)
  conv2 = Conv2D(32, (3, 3), activation='relu')(pool1)
  pool2 = MaxPooling2D((2, 2))(conv2)
  flatten = Flatten()(pool2)
  output = flatten
  # Classifier
  #hidden1 = Dense(64, activation='relu')(flatten)
  #dropout1 = Dropout(0.2)(hidden1)
  #hidden2 = Dense(32, activation='relu')(hidden1)
  #dropout2 = Dropout(0.2)(hidden2)
  #output = Dense(9, activation='softmax')(hidden2)

  model = Model(inputs= input, outputs=output)
  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
def PS2W(input_shape1,input_shape2,input_shape3):
  input_p = Input(input_shape1)
  input_a = Input(input_shape2)
  # encoded representation of input
  conv1_p = Conv2D(128, (3, 3), activation='relu',padding = 'same')(input_p)
  pool1_p = MaxPooling2D((2, 2))(conv1_p)
  conv2_p = Conv2D(64, (3, 3), activation='relu',padding = 'same')(pool1_p)
  pool2_p = MaxPooling2D((2, 2))(conv2_p)
  conv3_p = Conv2D(32, (3, 3), activation='relu',padding = 'same')(pool2_p)
  pool3_p = MaxPooling2D((2, 2))(conv3_p)

  conv1_a = Conv2D(128, (3, 3), activation='relu',padding = 'same')(input_a)
  pool1_a = MaxPooling2D((2, 2))(conv1_a)
  conv2_a = Conv2D(64, (3, 3), activation='relu',padding = 'same')(pool1_a)
  pool2_a = MaxPooling2D((2, 2))(conv2_a)
  conv3_a = Conv2D(32, (3, 3), activation='relu',padding = 'same')(pool2_a)
  pool3_a = MaxPooling2D((2, 2))(conv3_a)

  flatten_p = Flatten()(pool3_p)
  flatten_a = Flatten()(pool3_a)
  combined = concatenate([flatten_p,flatten_a])
  encoded = Dense(1000, activation='relu')(combined)
  # decoded representation of code
  decoded1 = Dense(np.prod(input_shape3), activation='relu')(encoded)
  #decoded2 = Dense(600, activation='relu')(decoded1)
  # Model which take input image and shows decoded images
  autoencoder = Model([input_p,input_a], decoded1)

  autoencoder.compile(optimizer='adam', loss='mse')
  return autoencoder

def P2W():
  input_p = Input(shape=(100,21,1))
  # encoded representation of input
  conv1_p = Conv2D(128, (3, 3), activation='relu',padding = 'same')(input_p)
  pool1_p = MaxPooling2D((2, 2))(conv1_p)
  conv2_p = Conv2D(64, (3, 3), activation='relu',padding = 'same')(pool1_p)
  pool2_p = MaxPooling2D((2, 2))(conv2_p)
  conv3_p = Conv2D(32, (3, 3), activation='relu',padding = 'same')(pool2_p)
  pool3_p = MaxPooling2D((2, 2))(conv3_p)

  flatten_p = Flatten()(pool3_p)

  encoded = Dense(1000, activation='relu')(flatten_p)
  # decoded representation of code
  decoded1 = Dense(600, activation='relu')(encoded)
  # Model which take input image and shows decoded images
  autoencoder = Model(input_p, decoded1)

  autoencoder.compile(optimizer='adam', loss='mse')
  return autoencoder

def S2W():
  input_p = Input(shape=(25,32,1))
  # encoded representation of input
  conv1_p = Conv2D(128, (3, 3), activation='relu',padding = 'same')(input_p)
  pool1_p = MaxPooling2D((2, 2))(conv1_p)
  conv2_p = Conv2D(64, (3, 3), activation='relu',padding = 'same')(pool1_p)
  pool2_p = MaxPooling2D((2, 2))(conv2_p)
  conv3_p = Conv2D(32, (3, 3), activation='relu',padding = 'same')(pool2_p)
  pool3_p = MaxPooling2D((2, 2))(conv3_p)

  flatten_p = Flatten()(pool3_p)

  encoded = Dense(1000, activation='relu')(flatten_p)
  # decoded representation of code
  decoded1 = Dense(600, activation='relu')(encoded)
  # Model which take input image and shows decoded images
  autoencoder = Model(input_p, decoded1)

  autoencoder.compile(optimizer='adam', loss='mse')
  return autoencoder

def W2P(input_shape=(20,30,1) ):
  input_p = Input(input_shape)
  # encoded representation of input
  conv1_p = Conv2D(128, (3, 3), activation='relu',padding = 'same')(input_p)
  pool1_p = MaxPooling2D((2, 2))(conv1_p)
  conv2_p = Conv2D(64, (3, 3), activation='relu',padding = 'same')(pool1_p)
  pool2_p = MaxPooling2D((2, 2))(conv2_p)
  conv3_p = Conv2D(32, (3, 3), activation='relu',padding = 'same')(pool2_p)
  pool3_p = MaxPooling2D((2, 2))(conv3_p)

  flatten_p = Flatten()(pool3_p)

  encoded = Dense(700, activation='relu')(flatten_p)
  encoded1 = Dense(1500, activation='relu')(encoded)
  # decoded representation of code
  decoded1 = Dense(2100, activation='relu')(encoded1)
  # Model which take input image and shows decoded images
  autoencoder = Model(input_p, decoded1)

  autoencoder.compile(optimizer='adam', loss='mse')
  return autoencoder